In [2]:
import re
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder, StandardScaler

## Feature engineering

In [3]:
df = pd.read_csv('data/clean_df.csv')

### Преобразование категориальных переменных

In [29]:
df.describe(include=[object])

,session_id,utm_source,utm_medium,device_category,device_os,device_brand,device_browser,geo_country,geo_city
count,1732266,1732266,1732266,1732266,1732266,1732266,1732266,1732266,1732266
unique,1732266,36,19,3,6,7,55,2,118
top,9055434745589932991.1637753792.1637753792,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Apple,Chrome,Russia,Moscow
freq,1,552555,525206,1368706,881466,526948,951573,1682423,750928


In [30]:
cat_variables = [category_var for category_var in df.describe(include=[object]).keys() if category_var != 'session_id']
cat_variables

['utm_source',
 'utm_medium',
 'device_category',
 'device_os',
 'device_brand',
 'device_browser',
 'geo_country',
 'geo_city']

In [31]:
for variable in cat_variables:
    print(
        f"Количество уникальных значений {df[variable].name}: {len(df[variable].unique())}"
    )

Количество уникальных значений utm_source: 36
Количество уникальных значений utm_medium: 19
Количество уникальных значений device_category: 3
Количество уникальных значений device_os: 6
Количество уникальных значений device_brand: 7
Количество уникальных значений device_browser: 55
Количество уникальных значений geo_country: 2
Количество уникальных значений geo_city: 118


Создаю дадафрейм только с необходимыми переменными

In [33]:
data = df[cat_variables]
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1732266 entries, 0 to 1732265
Data columns (total 8 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   utm_source       object
 1   utm_medium       object
 2   device_category  object
 3   device_os        object
 4   device_brand     object
 5   device_browser   object
 6   geo_country      object
 7   geo_city         object
dtypes: object(8)
memory usage: 105.7+ MB


Преобразую категориальные переменные с помощью OnrHotEncoder

In [34]:
ohe = OneHotEncoder(sparse_output=False)
ohe.fit(data[cat_variables])
ohe_data = ohe.transform(data)
print(ohe_data.shape)
ohe_data

(1732266, 246)


array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

Дополняю датафрейм перобразованными переменными

In [35]:
df[ohe.get_feature_names_out()] = ohe_data
df.head(5)

/var/folders/3n/9btqzdsn4pj6vt892qs385w80000gn/T/ipykernel_22636/3830920143.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ohe.get_feature_names_out()] = ohe_data
/var/folders/3n/9btqzdsn4pj6vt892qs385w80000gn/T/ipykernel_22636/3830920143.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[ohe.get_feature_names_out()] = ohe_data
/var/folders/3n/9btqzdsn4pj6vt892qs385w80000gn/T/ipykernel_22636/3830920143.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many

,session_id,visit_number,utm_source,utm_medium,device_category,device_os,device_brand,device_browser,geo_country,geo_city,...,geo_city_Volgograd,geo_city_Vologda,geo_city_Voronezh,geo_city_Vyborg,geo_city_Yaroslavl,geo_city_Yekaterinburg,geo_city_Yuzhno-Sakhalinsk,geo_city_Zheleznodorozhny,geo_city_Zhukovskiy,geo_city_other_city
0,9055434745589932991.1637753792.1637753792,1,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Huawei,Chrome,Russia,other_city,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,905544597018549464.1636867290.1636867290,1,MvfHsxITijuriZxsqZqt,cpm,mobile,Android,Samsung,Samsung Internet,Russia,Moscow,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,9055446045651783499.1640648526.1640648526,1,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Huawei,Chrome,Russia,Krasnoyarsk,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,9055447046360770272.1622255328.1622255328,1,kjsLglQLzykiRbcDiGcD,cpc,mobile,Android,Xiaomi,Chrome,Russia,Moscow,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,9055447046360770272.1622255345.1622255345,2,kjsLglQLzykiRbcDiGcD,cpc,mobile,Android,Xiaomi,Chrome,Russia,Moscow,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Преобразую количественные переменные

In [36]:
count_variables = [
    'visit_number',
    'month',
    'day',
    'hour',
    'screen_square',
]

In [37]:
columns_std = [f'{col}_std' for col in count_variables]

data_num = df[count_variables]
data_num

,visit_number,month,day,hour,screen_square
0,1,11,24,14,259200
1,1,11,14,8,328790
2,1,12,28,2,259200
3,1,5,29,5,308898
4,2,5,29,5,308898
...,...,...,...,...,...
1732261,1,12,22,10,521250
1732262,1,5,26,8,329160
1732263,1,11,15,15,250125
1732264,1,11,29,15,334443


Для преобразования количественных переменных использую StandartScaler

In [38]:
std_scaler = StandardScaler()
std_scaler.fit(data_num)
std_scaler_new = std_scaler.transform(data_num)
std_scaler_new


array([[-0.13981717,  0.77862821,  0.78515779,  0.06309159, -0.97586466],
       [-0.13981717,  0.77862821, -0.35226829, -0.91558632, -0.26300365],
       [-0.13981717,  1.23134646,  1.24012823, -1.89426423, -0.97586466],
       ...,
       [-0.13981717,  0.77862821, -0.23852568,  0.22620457, -1.06882649],
       [-0.13981717,  0.77862821,  1.35387084,  0.22620457, -0.20509586],
       [-0.13981717,  1.23134646,  1.58135605,  0.87865651,  1.70850419]])

Добавляю новые фичи в исходный датасет

In [39]:
df[columns_std] = std_scaler_new
print(df.columns.to_list())
df.head(5)

['session_id', 'visit_number', 'utm_source', 'utm_medium', 'device_category', 'device_os', 'device_brand', 'device_browser', 'geo_country', 'geo_city', 'target_action', 'screen_square', 'month', 'day', 'hour', 'utm_source_BHcvLfOaCWvWTykYqHVe', 'utm_source_BKeImrJuRDZcHiSSTdzm', 'utm_source_DnEUulZAecfGPvdtZBYS', 'utm_source_GpAkIXsclxDGyILfNlrR', 'utm_source_ISrKoXQCxqqYvAZICvjs', 'utm_source_IZEXUFLARCUMynmHNBGo', 'utm_source_KgicpPxiEQfzPlPwQZJq', 'utm_source_MvfHsxITijuriZxsqZqt', 'utm_source_PlbkrSYoHuZBWfYjYnfw', 'utm_source_QxAxdyPLuQMEcrdZWdWb', 'utm_source_RmEBuqrriAfAVsLQQmhk', 'utm_source_SzZERoLMmrEUEhDaYcyN', 'utm_source_TxKUcPpthBDPieTGmVhx', 'utm_source_YlsczTIyBSwTLNtuDkCd', 'utm_source_ZpYIoDJMcFzVoPFsHGJL', 'utm_source_aXQzDWsJuGXeBXexNHjc', 'utm_source_bByPQxmDaMXgpHeypKSM', 'utm_source_dyicZQGoeASogoSafjEh', 'utm_source_eLzNJHzPelJpEyBwMrKo', 'utm_source_fDLlAcSmythWSCVMvqvL', 'utm_source_fgymSoTvjKPEgaIJqsiH', 'utm_source_gDBGzjFKYabGgSPZvrDH', 'utm_source_gVRrcxiD

/var/folders/3n/9btqzdsn4pj6vt892qs385w80000gn/T/ipykernel_22636/749244879.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[columns_std] = std_scaler_new
/var/folders/3n/9btqzdsn4pj6vt892qs385w80000gn/T/ipykernel_22636/749244879.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[columns_std] = std_scaler_new
/var/folders/3n/9btqzdsn4pj6vt892qs385w80000gn/T/ipykernel_22636/749244879.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor 

,session_id,visit_number,utm_source,utm_medium,device_category,device_os,device_brand,device_browser,geo_country,geo_city,...,geo_city_Yekaterinburg,geo_city_Yuzhno-Sakhalinsk,geo_city_Zheleznodorozhny,geo_city_Zhukovskiy,geo_city_other_city,visit_number_std,month_std,day_std,hour_std,screen_square_std
0,9055434745589932991.1637753792.1637753792,1,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Huawei,Chrome,Russia,other_city,...,0.0,0.0,0.0,0.0,1.0,-0.139817,0.778628,0.785158,0.063092,-0.975865
1,905544597018549464.1636867290.1636867290,1,MvfHsxITijuriZxsqZqt,cpm,mobile,Android,Samsung,Samsung Internet,Russia,Moscow,...,0.0,0.0,0.0,0.0,0.0,-0.139817,0.778628,-0.352268,-0.915586,-0.263004
2,9055446045651783499.1640648526.1640648526,1,ZpYIoDJMcFzVoPFsHGJL,banner,mobile,Android,Huawei,Chrome,Russia,Krasnoyarsk,...,0.0,0.0,0.0,0.0,0.0,-0.139817,1.231346,1.240128,-1.894264,-0.975865
3,9055447046360770272.1622255328.1622255328,1,kjsLglQLzykiRbcDiGcD,cpc,mobile,Android,Xiaomi,Chrome,Russia,Moscow,...,0.0,0.0,0.0,0.0,0.0,-0.139817,-1.937681,1.353871,-1.404925,-0.466772
4,9055447046360770272.1622255345.1622255345,2,kjsLglQLzykiRbcDiGcD,cpc,mobile,Android,Xiaomi,Chrome,Russia,Moscow,...,0.0,0.0,0.0,0.0,0.0,-0.053926,-1.937681,1.353871,-1.404925,-0.466772


Удаляю лишние колонки (переменные без преобразования)

In [40]:
columns_for_drop = cat_variables.copy()
columns_for_drop.extend(count_variables)
columns_for_drop

['utm_source',
 'utm_medium',
 'device_category',
 'device_os',
 'device_brand',
 'device_browser',
 'geo_country',
 'geo_city',
 'visit_number',
 'month',
 'day',
 'hour',
 'screen_square']

In [41]:
df_prepared = df.drop(columns=columns_for_drop)
df_prepared

,session_id,target_action,utm_source_BHcvLfOaCWvWTykYqHVe,utm_source_BKeImrJuRDZcHiSSTdzm,utm_source_DnEUulZAecfGPvdtZBYS,utm_source_GpAkIXsclxDGyILfNlrR,utm_source_ISrKoXQCxqqYvAZICvjs,utm_source_IZEXUFLARCUMynmHNBGo,utm_source_KgicpPxiEQfzPlPwQZJq,utm_source_MvfHsxITijuriZxsqZqt,...,geo_city_Yekaterinburg,geo_city_Yuzhno-Sakhalinsk,geo_city_Zheleznodorozhny,geo_city_Zhukovskiy,geo_city_other_city,visit_number_std,month_std,day_std,hour_std,screen_square_std
0,9055434745589932991.1637753792.1637753792,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,-0.139817,0.778628,0.785158,0.063092,-0.975865
1,905544597018549464.1636867290.1636867290,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,-0.139817,0.778628,-0.352268,-0.915586,-0.263004
2,9055446045651783499.1640648526.1640648526,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.139817,1.231346,1.240128,-1.894264,-0.975865
3,9055447046360770272.1622255328.1622255328,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.139817,-1.937681,1.353871,-1.404925,-0.466772
4,9055447046360770272.1622255345.1622255345,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.053926,-1.937681,1.353871,-1.404925,-0.466772
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1732261,9055415581448263752.1640159305.1640159305,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.139817,1.231346,0.557673,-0.589360,1.708504
1732262,9055421130527858185.1622007305.1622007305,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.139817,-1.937681,1.012643,-0.915586,-0.259213
1732263,9055422955903931195.1636979515.1636979515,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.139817,0.778628,-0.238526,0.226205,-1.068826
1732264,905543020766873816.1638189404.1638189404,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,-0.139817,0.778628,1.353871,0.226205,-0.205096


In [42]:
print(df_prepared.shape)
df_prepared.columns.to_list()

(1732266, 253)


['session_id',
 'target_action',
 'utm_source_BHcvLfOaCWvWTykYqHVe',
 'utm_source_BKeImrJuRDZcHiSSTdzm',
 'utm_source_DnEUulZAecfGPvdtZBYS',
 'utm_source_GpAkIXsclxDGyILfNlrR',
 'utm_source_ISrKoXQCxqqYvAZICvjs',
 'utm_source_IZEXUFLARCUMynmHNBGo',
 'utm_source_KgicpPxiEQfzPlPwQZJq',
 'utm_source_MvfHsxITijuriZxsqZqt',
 'utm_source_PlbkrSYoHuZBWfYjYnfw',
 'utm_source_QxAxdyPLuQMEcrdZWdWb',
 'utm_source_RmEBuqrriAfAVsLQQmhk',
 'utm_source_SzZERoLMmrEUEhDaYcyN',
 'utm_source_TxKUcPpthBDPieTGmVhx',
 'utm_source_YlsczTIyBSwTLNtuDkCd',
 'utm_source_ZpYIoDJMcFzVoPFsHGJL',
 'utm_source_aXQzDWsJuGXeBXexNHjc',
 'utm_source_bByPQxmDaMXgpHeypKSM',
 'utm_source_dyicZQGoeASogoSafjEh',
 'utm_source_eLzNJHzPelJpEyBwMrKo',
 'utm_source_fDLlAcSmythWSCVMvqvL',
 'utm_source_fgymSoTvjKPEgaIJqsiH',
 'utm_source_gDBGzjFKYabGgSPZvrDH',
 'utm_source_gVRrcxiDQubJiljoTbGm',
 'utm_source_geDcueAOghDzHkGMmdOq',
 'utm_source_ghoaGAksqhKomdFrxgyJ',
 'utm_source_hTjLvqNxGggkGnxSCaTm',
 'utm_source_iNFgfQPqHPBuvGCYtr

In [43]:
df_prepared['target_action'].value_counts()

target_action
0    1681952
1      50314
Name: count, dtype: int64

Сохраняю финальный датасет в новый файл

In [44]:
df_prepared.to_csv('data/final_prepared.csv', index=False)